In [3]:
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup
from datetime import timedelta
from urllib.parse import urlparse

import warnings
warnings.filterwarnings('ignore')

fullpath = 'PATH/TO/YOUR/GOOGLEDATA/Wiedergabeverlauf.html'
soup = BeautifulSoup(open(fullpath), 'html.parser')
td = pd.Timedelta(6, unit='h')

### 1. Youtube Clicks

#### Store video data blocks in list video_containers

In [21]:
video_containers = soup.find_all("div", {"class": "content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1"})

Exemplary element of that list

In [22]:
video_containers[0]

<div class="content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1"><a href="https://www.youtube.com/watch?v=Kzdc_mbHJQQ">Improve Your Manual Skills | MTB Coaching With Neil</a> angesehen<br/><a href="https://www.youtube.com/channel/UC_A--fhX5gea0i4UtpD99Gg">Global Mountain Bike Network</a><br/>08.04.2020, 17:45:36 MESZ</div>

We can now obtain all the information separately 

In [23]:
# Lists to store the scraped data in
info = []
links = []

# Extract data from individual video container
for container in video_containers:
    # The name
    name = container.text
    info.append(name)
    
    if container.find('a') is not None:
        url = container.a.get('href')
    else:
        url = ""
    links.append(url)

In [24]:
df = pd.DataFrame({'text': info,
'Video_URL': links
})

A quick look on how our dataframe looks like right now reveals that we need to extract more info from the text column, which holds the TITLE, the CHANNEL and the DATE. Video_URL is already how it is supposed to be.

In [25]:
df.head()

,text,Video_URL
0,Improve Your Manual Skills | MTB Coaching With...,https://www.youtube.com/watch?v=Kzdc_mbHJQQ
1,Bunny Hop MTB Fahrtechnik Tutorial | Step by s...,https://www.youtube.com/watch?v=pDwqQKKXkQo
2,Hardcore Hardtail Mtb Abenteuer/ Tour durch de...,https://www.youtube.com/watch?v=frGH0jGbsd4
3,FAZIT Tubeless: 4 Monate ohne Schlauch im Moun...,https://www.youtube.com/watch?v=V4IaboqyB2s
4,How To Avoid Punctures On Your Mountain Bike a...,https://www.youtube.com/watch?v=duS4GUuMhR0


#### Text column split to extract Channel, Date und Titel

In [26]:
# Edits on Text column to extract CHANNEL, DATE and TITLE
df["Video_Title"] = df["text"].str.split("angesehen", n = 1, expand = True)[0]
df["Channel_Date"] = df["text"].str.split("angesehen", n = 1, expand = True)[1]

df["Channel_Day"] = df["Channel_Date"].str.split(", ", n=1, expand=True)[0]
df["Hour"] = df["Channel_Date"].str.split(", ", n=1, expand=True)[1]

df["Video_Channel"] = df["Channel_Day"].str[:-10]
df["Day"] = df["Channel_Day"].str[-10:]
df["Clicked_Date"] = df["Day"] + " " + df["Hour"]

Now we drop all the columns that we only needed for temporary variable storage and reorder the columns:

In [27]:
df = df.drop(axis=1, columns={"Hour", "Channel_Day", "text", "Channel_Date", "Day"})
df = df[["Video_Title", "Video_URL", "Video_Channel", "Clicked_Date"]]

#### Result

In [28]:
df.head(5)

,Video_Title,Video_URL,Video_Channel,Clicked_Date
0,Improve Your Manual Skills | MTB Coaching With...,https://www.youtube.com/watch?v=Kzdc_mbHJQQ,Global Mountain Bike Network,08.04.2020 17:45:36 MESZ
1,Bunny Hop MTB Fahrtechnik Tutorial | Step by s...,https://www.youtube.com/watch?v=pDwqQKKXkQo,Leo Kast,08.04.2020 16:59:30 MESZ
2,Hardcore Hardtail Mtb Abenteuer/ Tour durch de...,https://www.youtube.com/watch?v=frGH0jGbsd4,Ride it Official,08.04.2020 16:57:54 MESZ
3,FAZIT Tubeless: 4 Monate ohne Schlauch im Moun...,https://www.youtube.com/watch?v=V4IaboqyB2s,Fabio Schäfer,08.04.2020 16:54:48 MESZ
4,How To Avoid Punctures On Your Mountain Bike,https://www.youtube.com/watch?v=duS4GUuMhR0,Global Mountain Bike Network,08.04.2020 16:48:03 MESZ


Some videos have been deleted over the years. We can remove these rows by searching for the String "Ein Video" from the sentence "Ein Video, das entfernt wurde.". Also, deleted videos might have an empty value for Channel.

In [29]:
df = df[(df.Video_Title.str.contains("Ein Video") == False) & (df.Video_Channel != "")]

#### Extracting video id from URL
We do that so that we can call the youtube API later

In [30]:
def video_id(url):
    """
    Examples:
    - http://youtu.be/SA2iWivDJiE
    - http://www.youtube.com/watch?v=_oPAwA_Udwc&feature=feedu
    - http://www.youtube.com/embed/SA2iWivDJiE
    - http://www.youtube.com/v/SA2iWivDJiE?version=3&amp;hl=en_US
    """
    o = urlparse(url)
    if o.netloc == 'youtu.be':
        return o.path[1:]
    elif o.netloc in ('www.youtube.com', 'youtube.com'):
        if o.path == '/watch':
            id_index = o.query.index('v=')
            return o.query[id_index+2:id_index+13]
        elif o.path[:7] == '/embed/':
            return o.path.split('/')[2]
        elif o.path[:3] == '/v/':
            return o.path.split('/')[2]
    return None  # fail?

In [31]:
df["Video_ID"] = df["Video_URL"].apply(lambda x: video_id(x))

#### Formatting the Date

This is especially important to convert the timestamps from when I was in Canada

In [32]:
df["Clicked_Date"] = pd.to_datetime(df["Clicked_Date"].str[:-5], format='%d.%m.%Y %H:%M:%S', errors="coerce")

In [33]:
df["Year"] = pd.DatetimeIndex(df["Clicked_Date"]).year
df["Month"] = pd.DatetimeIndex(df["Clicked_Date"]).month
df["Day of Week"] = df["Clicked_Date"].dt.day_name()

Canadian time is 6 hours behind European Time

In [34]:
df_canada = df[(df["Clicked_Date"] > pd.to_datetime("28.08.2016", format="%d.%m.%Y"))
  & (df["Clicked_Date"] < pd.to_datetime("29.12.2016", format="%d.%m.%Y"))]

df_rest = df[(df["Clicked_Date"] <= pd.to_datetime("28.08.2016", format="%d.%m.%Y"))
  | (df["Clicked_Date"] >= pd.to_datetime("29.12.2016", format="%d.%m.%Y"))]

df_canada["Clicked_Date"] = df_canada["Clicked_Date"] - td

Putting it back together:

In [35]:
df = df_rest.append(df_canada)

Export

In [36]:
df.to_csv("EXPORT/DIRECTORY/youtube_clicks.csv", index = False)

__________

### 2. Youtube Search

In [37]:
fullpath = 'PATH/TO/YOUR/GOOGLEDATA/Suchverlauf.html'
soup = BeautifulSoup(open(fullpath), 'html.parser')

In [38]:
search_containers = soup.find_all("div", {"class": "content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1"})

In [39]:
search_containers[0]

<div class="content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1">Nach <a href="https://www.youtube.com/results?search_query=hardtail+mtb+techniques">hardtail mtb techniques</a> gesucht<br/>08.04.2020, 16:57:34 MESZ</div>

In [46]:
# Lists to store the scraped data in
info = []

# Extract data from individual container
for container in search_containers:
    # The name
    name = container.text
    info.append(name)

In [47]:
df = pd.DataFrame({'text': info})

In [48]:
df.head(2)

,text
0,Nach hardtail mtb techniques gesucht08.04.2020...
1,"Nach mtb hardtail tour gesucht08.04.2020, 16:5..."


In [49]:
# Edits on Text column to extract SEARCH TERM and DATE
df["Search"] = df["text"].str.split("gesucht", n = 1, expand = True)[0].str[+5:]
df["Date"] = df["text"].str.split("gesucht", n = 1, expand = True)[1]

df = df.drop(axis=1, columns={"text"})

In [50]:
df.head(2)

,Search,Date
0,hardtail mtb techniques,"08.04.2020, 16:57:34 MESZ"
1,mtb hardtail tour,"08.04.2020, 16:57:15 MESZ"


In [51]:
df["Search_Date"] = pd.to_datetime(df["Date"].str[:-5], format='%d.%m.%Y, %H:%M:%S', errors="coerce")
df = df.drop(axis=1, columns={"Date"})

In [52]:
df_canada = df[(df.Search_Date > pd.to_datetime("28.08.2016", format="%d.%m.%Y"))
  & (df.Search_Date < pd.to_datetime("29.12.2016", format="%d.%m.%Y"))]

df_rest = df[(df.Search_Date <= pd.to_datetime("28.08.2016", format="%d.%m.%Y"))
  | (df.Search_Date >= pd.to_datetime("29.12.2016", format="%d.%m.%Y"))]

df_canada.Search_Date = df_canada.Search_Date - td


In [120]:
df.to_csv("EXPORT/DIRECTORY/youtube_search.csv", index = False)

### 3. Searched before Click

We can enrich our dataset by checking if I searched before watching a video or if it came up through autoplay or recommendations. We simply define a timedelta of 30 seconds and if I had done a youtube search within that delta before watching a video, that video is directly linked to that search term.

In [57]:
search = pd.read_csv("PATH/youtube_search.csv")
clicks = pd.read_csv("PATH/youtube_clicks.csv")

margin = datetime.timedelta(seconds = 30)

In [58]:
def searched_for(x):
    bol = False
    
    sd = search["Search_Date"].unique()
    search_dates = [pd.Timestamp(i) for i in sd]
    search_dates.sort()
    
    
    index = np.searchsorted(search_dates, pd.Timestamp(x))
    
    try:
        if pd.Timestamp(search_dates[index-1]) >= (pd.Timestamp(str(x)) - margin):
            bol = True
            return bol
        return bol
    except:
        return bol

In [ ]:
clicks["watched_after_search"] =  clicks["Clicked_Date"].apply(lambda x: searched_for(x))

In [ ]:
clicks["watched_after_search"].value_counts()

In [ ]:
clicks = clicks.dropna()

Final Dataset:

In [ ]:
clicks.head(5)

Export

In [126]:
clicks.to_csv("EXPORT/DIRECTORY/youtube_clicks.csv", index = False)